# Haarcascade wala code (kamjor)
 galat detections not accurate

In [1]:
import cv2
import numpy as np

In [2]:
def det(url):
    # Load pre-trained LBP classifier for human detection
    lbp_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_fullbody.xml")

    cap = cv2.VideoCapture(url)
    if not cap.isOpened():
        print("Error opening video file")
        return

    # Parameters for optical flow
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Read the first frame
    ret, old_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        cap.release()
        return

    old_frame = cv2.resize(old_frame, (512, 512))
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(old_frame)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (512, 512))
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect humans using LBP cascade
        humans = lbp_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Draw bounding boxes for detected humans
        for (x, y, w, h) in humans:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)


        cv2.imshow('Human Detection and Optical Flow', frame)

        old_gray = gray_frame.copy()

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Hog descriptor function better than haarcascade

In [2]:
def det(url):
    # Load pre-trained LBP classifier for human detection
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    cap = cv2.VideoCapture(url)
    if not cap.isOpened():
        print("Error opening video file")
        return

    # Parameters for optical flow
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Read the first frame
    ret, old_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        cap.release()
        return

    old_frame = cv2.resize(old_frame, (512, 512))
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(old_frame)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect humans using LBP cascade
        (rects, weights) = hog.detectMultiScale(frame, winStride=(4, 4),
                                            padding=(8, 8), scale=1.05)

        # Draw bounding boxes for detected humans
        for (x, y, w, h) in rects:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        #frame=cv2.resize(frame,(512,512))


        cv2.imshow('Human Detection and Optical Flow', frame)

        old_gray = gray_frame.copy()

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# link for videos (testing) 

In [3]:
link="/home/nirnaya/Desktop/Edge AI/Project2/src/2785536-uhd_2160_3840_25fps.mp4"

In [11]:
link="/home/nirnaya/Desktop/Edge AI/Project2/src/2791669-uhd_2160_3840_25fps.mp4"

In [8]:
link="/home/nirnaya/Desktop/Edge AI/Project2/src/8045821-hd_1080_1920_25fps.mp4"

# Functionj call

In [4]:
det(link)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObjec

# Camera ma chalauna

In [5]:
import cv2

# Initialize the HOG descriptor with the default people detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Open the video file or webcam
video_path = "/home/nirnaya/Desktop/Edge AI/Project2/src/8045821-hd_1080_1920_25fps.mp4"  # Replace with 0 for webcam or your video file path
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to half its original dimensions
    frame = cv2.resize(frame, (frame.shape[1] // 4, frame.shape[0] // 4))

    # Detect people in the resized frame
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    boxes, weights = hog.detectMultiScale(
        gray, winStride=(8, 8), padding=(16, 16), scale=1.05
    )

    # Draw rectangles around detected people
    for (x, y, w, h) in boxes:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the frame with detections
    cv2.imshow("Human Detection", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObject::moveToThread: Current thread (0x74e76509ff80) is not the object's thread (0x75ecc90).
Cannot move to target thread (0x74e76509ff80)

QObjec

In [18]:

cv2.ocl.setUseOpenCL(False)


In [5]:
import cv2

# Disable OpenCL to prevent memory issues
cv2.ocl.setUseOpenCL(False)

# Initialize the HOG descriptor with the default people detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Open the webcam (replace 0 with the appropriate camera index if needed)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open the webcam.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read the frame.")
        break

    # Resize the frame to half its original dimensions
    #frame = cv2.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2))

    # Detect people in the resized frame
    boxes, weights = hog.detectMultiScale(
        frame, winStride=(8, 8), padding=(16, 16), scale=1.05
    )

    # Draw rectangles around detected people
    for (x, y, w, h) in boxes:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the frame with detections
    cv2.imshow("Human Detection", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread

In [6]:
import cv2
import numpy as np

# Initialize the video capture object
cap = cv2.VideoCapture(0)  # Use 0 for default camera, or replace with the camera index

# Check if the camera is opened
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Read the first frame and convert it to grayscale
ret, frame1 = cap.read()
if not ret:
    print("Error: Could not read frame.")
    cap.release()
    exit()

prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Loop to read frames from the camera
while True:
    # Capture the next frame
    ret, frame2 = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert the frame to grayscale
    next_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate dense optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute the magnitude and angle of the 2D vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Use the angle to set the hue and normalize the magnitude
    hsv = np.zeros_like(frame1)
    hsv[..., 1] = 255
    hsv[..., 0] = angle * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    # Convert HSV to BGR
    rgb_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Display the result
    cv2.imshow('Optical Flow', rgb_flow)

    # Update the previous frame
    prev_gray = next_gray

    # Break the loop if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread (0x41286e0).
Cannot move to target thread (0x3213670)

QObject::moveToThread: Current thread (0x3213670) is not the object's thread

###### 